In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: /home/jovyan/work/FlareSense


In [2]:
import torch
import mlflow
import dagshub
import itertools
import torchmetrics
import src.utils.data15min as data
import pytorch_lightning as pl

from huggingface_hub import snapshot_download
from torchvision import transforms
from tqdm.notebook import tqdm
from src.models.CustomCNN import CustomCNN2

mlflow.pytorch.autolog()
torch.set_float32_matmul_precision("high")

2024/01/05 08:38:16 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.


2024-01-05 08:38:17.116821: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 08:38:17.227455: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 08:38:17.227489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 08:38:17.244427: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 08:38:17.279389: I tensorflow/core/platform/cpu_feature_guar

In [3]:
DATA_FOLDER_PATH = "data/raw/exported/"
INSTRUMENTS = ["Australia-ASSA_02", "Australia-ASSA_62"]
NUM_WORKERS = 12

In [4]:
# download needed files
snapshot_download(
    "StellarMilk/ecallisto-bursts",
    repo_type="dataset",
    allow_patterns=[f"{instrument}.zip" for instrument in INSTRUMENTS] + ["metadata.csv"],
    local_dir=DATA_FOLDER_PATH,
    revision="main",
)

for instrument in INSTRUMENTS:
    # if data available, skip
    if os.path.exists(f"{DATA_FOLDER_PATH}{instrument}"):
        print(f"Skipping {instrument}")
        continue
    
    # unzip if needed
    print(f"Unzipping {instrument}")
    !unzip -q {DATA_FOLDER_PATH}{instrument}.zip -d {DATA_FOLDER_PATH}

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Australia-ASSA_62.zip:   0%|          | 0.00/16.6G [00:00<?, ?B/s]

Australia-ASSA_02.zip:   0%|          | 0.00/5.01G [00:00<?, ?B/s]

metadata.csv:   0%|          | 0.00/19.4M [00:00<?, ?B/s]

Skipping Australia-ASSA_02
Skipping Australia-ASSA_62


In [5]:
lst_lr = [1e-3]
lst_weight_decay = [1e-2, 1e-3]
lst_dropout_p = [0.5]

data_module = data.ECallistoDataModule(
    data_folder=DATA_FOLDER_PATH,
    batch_size=64,
    num_workers=NUM_WORKERS,
    val_ratio=0.15,
    test_ratio=0.15,
    img_size=(193, 225),
    use_augmented_data=True,
    filter_instruments=["Australia-ASSA_02", "Australia-ASSA_62"],
    seed=0,
)
data_module.setup()

for current_lr, current_weight_decay, current_dropout_p in itertools.product(lst_lr, lst_weight_decay, lst_dropout_p):
    model = CustomCNN2(lr=current_lr, weight_decay=current_weight_decay, dropout_p=current_dropout_p)

    dagshub.init("FlareSense", "FlareSense", mlflow=True)
    mlflow.start_run()

    mlflow.log_params({
        "model": "CustomCNN2",
        "dropout_p": current_dropout_p,
        "batch_size": data_module.batch_size,               
        "val_ratio": data_module.val_ratio,
        "test_ratio": data_module.test_ratio,
        "use_data_augmentation": data_module.use_augmented_data,
        "filter_instruments": data_module.filter_instruments,
    })

    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_{run_id}")

    trainer = pl.Trainer(max_epochs=50, log_every_n_steps=1)

    trainer.fit(
        model,
        train_dataloaders=data_module.train_dataloader(),
        val_dataloaders=data_module.val_dataloader(),
    )

    trainer.test(model, dataloaders=data_module.test_dataloader())

    mlflow.end_run()

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=f44e76bc-189a-410f-881f-45bc45bb575d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=4d9f090ce0059bcb9ceae482646ab0b2833104d5ceadd173013fe7e218d2c7ae




Repository initialized!

GPU available: True (cuda), used: True


...

Validation: |                                                                   | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                                      | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.15588709712028503    │
│      test_precision       │    0.5339673757553101     │
│        test_recall        │    0.9515738487243652     │
└───────────────────────────┴───────────────────────────┘